# Introduction

This notebook allows you to run a simplified version of the ANTARESS workflow, for the following purpose:
- extracting from a CCF dataset the stellar lines occulted by a transiting planet
- performing a Rossiter-McLaughlin Revolutions analysis of the dataset

The default version of this notebook is ran with a mock dataset generated with ANTARESS (see `ANTARESS_nbook_mock.ipynb`), and you can modify the settings to analyze a series of measured CCF fits files.

To use the workflow to its full capabilities, set up a copy of the configuration file `ANTARESS_settings.py` and run `ANTARESS_launcher()` (see README for details). 

In [1]:
import ANTARESS_nbook_bground
input_nbook = ANTARESS_nbook_bground.init('RMR')

# System parameters

**Parameters of the host star**

Run this cell to define the host star. 
- `star_name` (string): Name of the star. Should not contain spaces/dashes.
- `Rs` (float): Radius of the star, in $R_\odot$.
- `vsini` (float): Projected stellar surface velocity, in km/s. The value given here is only used as guess for the fits, as it will eventually be derived from your analysis.
- `ld_u1` (float): First coefficient of the quadratic limb-darkening.
- `ld_u2` (float): Second coefficient of the quadratic limb-darkening.

In [2]:
input_nbook['par'].update({
    'star_name' : 'TOI178',
    'Rs'        : 0.651,
    'vsini'     : 1.5,
    'ld_u1'     : 0.787,
    'ld_u2'     : -0.006
})
ANTARESS_nbook_bground.init_star(input_nbook)

**Parameters of transiting planet**

Run this cell to set the transiting planet on which the RM analysis is applied. 
- `planet_name` (string): Name of the transiting planet. Should not contain spaces/dashes.
- `period` (float): Orbital period, in days.
- `T0` (float): Mid-transit time, in ${\rm BJD}_{\rm TBD}$.
- `ecc` (float): Orbital eccentricity.
- `long_per` (float): Longitude of periastron, in degrees. If the orbit is circular, fill in `90.`.
- `Kstar` (float): RV semi-amplitude, in m/s.
- `aRs` (float): Scaled separation $a/R_\star$.
- `incl` (float): Orbital inclination, in degrees.
- `RpRs` (float): Planet-to-star radius ratio $R_{\rm p}/R_\star$.

In [3]:
input_nbook['par'].update({
    'planet_name' : 'TOI178d',
    'period'      : 6.557700,
    'T0'          : 2458747.14623,
    'ecc'         : 0.,
    'long_per'    : 90.,
    'Kstar'       : 0.,
    'aRs'         : 19.57,
    'incl'        : 88.58,
    'RpRs'        : 0.03623
})
ANTARESS_nbook_bground.init_pl(input_nbook,'main')

**Parameters of additional planets in the system**

Run this cell to set additional planets, which will only contribute to the computation of the star Keplerian motion. 

The names and definitions of the parameters are the same as for the main planet (`aRs`, `incl`, `RpRs` are not required).

Copy the cell to add further planets

In [4]:
input_nbook['par'].update({
    'planet_name' : 'Numenor',
    'period'      : 8.,
    'T0'          : 2457176.2,
    'ecc'         : 0.,
    'long_per'    : 90.,
    'Kstar'       : 20.
})
#ANTARESS_nbook_bground.init_pl(input_nbook,'sec')

# Dataset properties

**Visit**

Run this cell to define the visit to be processed. It must overlap with the transit of the `transiting planet`.
- `instrument` (string): Instrument of the visit (e.g. `ESPRESSO`, `NIRPS_HE`...).
- `night` (string): Night of the visit. It is preferred to use the standard format, i.e. `YYYYMMDD`.
- `data_dir` (string): Path to the directory holding the observed .fits file (here, set to `mock` to retrieve an example dataset pre-generated with ANTARESS and stored in `Ongoing/ValinorRMR_Saved_data/`)

In [5]:
input_nbook['par'].update({
    'instrument'  : 'ESPRESSO',
    'night'       : 'Test_night',
    'data_dir'    : 'mock' 
})
ANTARESS_nbook_bground.add_vis(input_nbook)

**Systemic velocity**

Run this cell to set the systemic radial velocity `gamma` (float, in km/s).

Run the notebook a first time with `gamma` = 0 km/s to determine the value specific to your chosen visit (`gamma` can vary slightly with the epoch and the instrument, and should thus be derived from the dataset itself rather than set to an external value). Go directly to the `Running ANTARESS` section and read the value of `gamma` printed in the log.

Then, set `gamma` to the derived value and proceed with the analysis.

In [6]:
input_nbook['par']['gamma'] = 0.   
ANTARESS_nbook_bground.set_sysvel(input_nbook)

# Processing

**Alignment in star rest frame**

Run this cell to align the disk-integrated CCFs in the star rest frame. 

CCFs are originally defined in the Sun barycentric rest frame. This module shifts them by the systemic rv you defined above, and by the Keplerian rv motion induced by the planets you set up for this system.

In [7]:
ANTARESS_nbook_bground.align_prof(input_nbook)

**Flux scaling**

Run this cell to scale the disk-integrated CCFs to their correct relative flux level over the transit. 

The scaling depends on the limb-darkening coefficients and planet-to-star radius ratio you entered in the `System parameters`.

In [8]:
ANTARESS_nbook_bground.flux_sc(input_nbook)
ANTARESS_nbook_bground.DImast_weight(input_nbook)

**Residual CCFs**

Run this cell to extract residual CCFs, which are the difference between the master out-of-transit CCF (representative of the unocculted star) and the CCF in each exposure. During transit, the residual CCFs correspond to the light coming from the planet-occulted region of the stellar photosphere. 

In [9]:
ANTARESS_nbook_bground.extract_res(input_nbook)

**Intrinsic CCFs**

Run this cell to convert in-transit residual CCFs into intrinsic CCFs, which are corrected for the planetary occultation and only trace variations in the spectral profile of the stellar line along the transit chord.

The intrinsic CCF series can be retrieved under `/ANTARESS/Ongoing/planet_Saved_data/Intr_data/instrument_night_i.npz`, where `i` refers to in-transit indexes.  

In [10]:
ANTARESS_nbook_bground.extract_intr(input_nbook)

# Analysis and plot display

**Analysis of disk-integrated CCFs**

Run this cell to perform a Gaussian fit to each disk-integrated CCF. This will return the rv, FWHM, and contrast time-series of the average line from the entire stellar disk, which you can visualize below. This step is used to measure the systemic rv, to check that your Keplerian model matches the data, and to search for systematic trends in the line properties. 

Additional model and fit options are available through the configuration file. The full workflow will also allow you to correct for putative line trends.

In [11]:
ANTARESS_nbook_bground.ana_prof(input_nbook,'DI')

**Plotting disk-integrated CCFs**

Run this cell to plot the disk-integrated CCF series with their best fits. Uncomment `y_range` to adjust the flux range of the CCFs (vertical axis).

In [12]:
#input_nbook['par']['y_range'] = [2.8e5,3.55e5] 
ANTARESS_nbook_bground.plot_prof(input_nbook,'DI_prof')

**Plotting disk-integrated line properties**

Run this cell to plot the time-series of properties derived from the fits to disk-integrated CCFs. 

In [13]:
ANTARESS_nbook_bground.plot_prop(input_nbook,'DI')

**Analysis of intrinsic CCFs**

Run this cell to perform a Gaussian fit to each intrinsic CCF. This will return the rv, FWHM, and contrast time-series of the average stellar line from regions occulted along the transit chord, which you can visualize below. 

Intrinsic CCFs are usually measured with low S/N. In that case, you can run the fits using a MCMC (`fit_mode = "MCMC"`) rather than the default least-square minimization (`fit_mode = "chi2"`). To overwrite the default priors, you can further uncomment the `priors` field and define the lower and upper boundary of the uniform prior ranges. Look at the time-series of properties plotted below to adjust the priors, or at the MCMC chains and corner plots in the directories `/ANTARESS/Ongoing/star_Saved_data/Introrig_prop/instrument_night_mcmc/iexp*/`. Since running the MCMC may take some time, you can set `calc_fit` to `False` once the MCMC is done and you only want to manipulate the plots.

This step is used to identify which exposures should be included in the RM Revolutions fit, by assessing the quality of intrinsic CCFs. The derived stellar line properties can further be analyzed in the next step. Additional stellar line models and fit options are available through the configuration file. 

In [14]:
#Uncomment to use the MCMC
input_nbook['par']['fit_mode'] = 'mcmc'

#Uncomment to set your own priors
input_nbook['par']['priors']={        
    'rv':{'low':-5.,'high':5.},    
    'FWHM':{'low':0.,'high':20.},
    'ctrst':{'low':0.1,'high':1.}}

#Set to False to retrieve the fit results
input_nbook['par']['calc_fit'] = False

#ANTARESS_nbook_bground.ana_prof(input_nbook,'Intr')

**Plotting intrinsic CCFs**

Run this cell to plot the intrinsic CCF series with their best fits. Uncomment `y_range` to adjust the flux range of the CCFs (vertical axis). 

In [15]:
#input_nbook['par']['y_range'] = [2.8e5,3.55e5] 
ANTARESS_nbook_bground.plot_prof(input_nbook,'Intr_prof')

**Analysis of intrinsic properties**

Run this cell to fit the series of rv, contrast, and FWHM derived from the fits to individual intrinsic CCFs. This step is used to determine the best models describing variations of these properties along the transit chord, which will then be used in the global Rossiter-McLaughlin Revolutions fit. 

The models are controlled by various parameters `x`, for which you must define a range [${\rm x}_{\rm low}$,${\rm x}_{\rm high}$] in the `mod_prop` dictionary. If you run a least-square fit, it will be initialized with the mean of this range. If you run a MCMC fit, its walkers will be initialized over this range. If the data allows it you can increase the complexity of the models by uncommenting additional parameters, as described below.

- The intrinsic rv series traces the motion of the stellar surface along the transit chord, and thus depend on the sky-projected angle `lambda` (in degrees) between the stellar spin and orbital normal, and on the sky-projected stellar rotational velocity `veq` (in km/s). Here we are assuming solid-body rotation for the star, so that the stellar inclination is degenerate and fixed to 90$^{\circ}$. The rv model can be modulated by stellar differential rotation, in which case you can break this degeneracy and fit for `istar` (in degrees) and the coefficient `alpha` (0 corresponds to solid rotation, 1 to the poles not rotating). The rv model can further be modulated by convective blueshift, defined as a polynomial of the center-to-limb angle for which you can control the linear or quadratic coefficients `c1_CB` and `c2_CB`.

- The intrinsic contrast and FWHM series trace the shape of the planet-occulted stellar lines along the transit chord. They are modelled as polynomials of the center-to-limb angle for which you can control the coefficients `x_i`, with `i`$\ge0$. 

Depending on the number of parameters and the dataset, you may need to adjust the number of walkers in the MCMC, their steps, and the burn-in phase by uncommenting the `mcmc_set` dictionary. Check the MCMC chains in `/ANTARESS/Ongoing/star_Saved_data/Joined_fits/IntrProp/mcmc/x/` to adjust these numbers. As with the fits to intrinsic CCFs you can overwrite the default MCMC priors for any of the fitted parameters. Once you have run the fit, you can set `calc_fit` to `False` to manipulate the plots. 

You can fit all in-transit exposures by setting `idx_in_fit` to `all`. However the occulted stellar lines and their derived properties may be so noisy in some exposures, especially at the stellar limbs, that they do not bring any constraint to the fits. It also happens that the planet only occults the star for a small fraction of the ingress/egress exposure, as in the example mock dataset, so that the model planet does not transit. In that case, you can change `idx_in_fit` to the list of indexes indicating which exposures to fit (indexes are counted from 0 from the first in-transit exposure). 

If the star is too faint or the planet too small, the properties may all be derived with a precision that is too low to analyze them here. In that case, you can apply directly the Revolutions fit with the simplest models to describe these properties. 

Finally, in this notebook we are using a simple Gaussian profile to fit the intrinsic lines. Although this model includes instrumental convolution, it is an approximation compared to using ANTARESS numerical stellar grid to simulate realistic line profiles, as is done when applying the RM Revolutions fit. Here we do not account in particular for the blurring induced by the planet motion, which is significant for long exposure times and fast-rotators. Use the configuration file if you want to fit more finely individual intrinsic profiles. 

In [16]:
#Exposures to be fitted
input_nbook['par']['idx_in_fit'] = range(1,25)

#Uncomment to use the MCMC
input_nbook['par']['fit_mode'] = 'mcmc'

#Uncomment to adjust the MCMC chains (default nwalkers = 50, nsteps = 1000, nburn = 200)
#input_nbook['par']['mcmc_set']={'nwalkers':10,'nsteps':100,'nburn':0} 

#Surface RV: main parameters
input_nbook['par']['mod_prop']={ 
    'veq':[3.,5.],
    'lambda':[-5.,5.]    
}

#Surface RV: convective blueshift
#input_nbook['par']['mod_prop']['c1_CB']=[-1.,1.] 
#input_nbook['par']['mod_prop']['c2_CB']=[-1.,1.]

#Surface RV: differential rotation
#input_nbook['par']['mod_prop']['alpha']=[0.,0.2] 
#input_nbook['par']['mod_prop']['istar']=[-90.,90.]

#Intrinsic line contrast and FWHM: polynomial coefficients
input_nbook['par']['mod_prop']['contrast_0']=[0.2,0.7]   
#input_nbook['par']['mod_prop']['contrast_1']=[-1.,1.]   
input_nbook['par']['mod_prop']['FWHM_0']=[5.,10.]   
#input_nbook['par']['mod_prop']['FWHM_1']=[-5.,5.]

#Uncomment to set your own priors
input_nbook['par']['priors']={        
    'veq':{'low':0.,'high':100.},  
    'lambda':{'low':-180.,'high':180.},  
    'contrast_0':{'low':0.,'high':1.},    
    'FWHM_0':{'low':0.,'high':20.}}

#Set to False to retrieve the fit results
input_nbook['par']['calc_fit'] = False

#ANTARESS_nbook_bground.ana_jointprop(input_nbook,'Intr')

**Plotting intrinsic properties**

Run this cell to plot the intrinsic CCF properties and their best-fit models.

In [17]:
#ANTARESS_nbook_bground.plot_prop(input_nbook,'Intr')

**Rossiter-McLaughlin Revolutions**

Run this cell to perform a RM Revolution fit to your dataset. A joint model for the occulted line profiles along the transit chord will be fitted to the exposures that you can select with `idx_in_fit`, based on your analysis from the previous step.

The model is controlled by the same parameters as for individual intrinsic properties, so that you can define the model here in the same way using the parameters that you identified as relevant in the previous step.

It is advised to run the fit through a MCMC, which you can set up after a preliminary run by checking the MCMC chains in `/ANTARESS/Ongoing/star_Saved_data/Joined_fits/IntrProf/mcmc/`.

Information and results from the fit can be found in the file `/ANTARESS/Ongoing/star_Saved_data/Joined_fits/IntrProf/mcmc/Outputs`.

The notebook assumes a simple Gaussian profile to fit intrinsic lines, with default properties for ANTARESS stellar grid. Additional options are available through the configuration file. 

In [18]:
#Exposures to be fitted
input_nbook['par']['idx_in_fit'] = 'all'

#Uncomment to use the MCMC
input_nbook['par']['fit_mode'] = 'mcmc'

#Uncomment to adjust the MCMC chains (default nwalkers = 3 x number of free parameters, nsteps = 5000, nburn = 1000)    
input_nbook['par']['mcmc_set']={'nwalkers':20,'nsteps':1000,'nburn':200} 

#Model parameters
input_nbook['par']['mod_prop']={ 
    'veq':[0.5,3.],
    'lambda':[-180.,180.],
    'contrast_0':[0.4,0.9],    
    'FWHM_0':[4.,12.]}   

#Uncomment to set your own priors
input_nbook['par']['priors']={        
    'veq':{'low':0.,'high':5.},  
    'lambda':{'low':0.,'high':180.},  
    'contrast_0':{'low':0.3,'high':1.},    
    'FWHM_0':{'low':0.,'high':20.}}

#Set to False to retrieve the fit results
input_nbook['par']['calc_fit'] = True

ANTARESS_nbook_bground.ana_jointprof(input_nbook,'Intr')

In [19]:

#ANTARESS_nbook_bground.loc_prof_corr(input_nbook)

**Plotting intrinsic CCF maps**

XXX

In [20]:
input_nbook['par']['v_range'] = None # [0.6,1.1]
#ANTARESS_nbook_bground.plot_map(input_nbook,'Intr_prof')
#ANTARESS_nbook_bground.plot_map(input_nbook,'Intr_prof_est')
#ANTARESS_nbook_bground.plot_map(input_nbook,'Intr_prof_res')

# Running ANTARESS

Run this cell to run the ANTARESS workflow.

In [21]:
%cd ../Method/ 
from ANTARESS_launch.ANTARESS_launcher import ANTARESS_launcher
ANTARESS_launcher(nbook_dic = input_nbook)

/Users/bourrier/Travaux/ANTARESS/Method


/Users/bourrier/opt/anaconda3/lib/python3.8/site-packages/pyfits/__init__.py:21: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  warnings.warn('PyFITS is deprecated, please use astropy.io.fits',


****************************************
Launching ANTARESS
****************************************

Multi-threading: 8 threads available
Running with artificial data
Covariances discounted
Accounting for Keplerian motion from all planets
Automatic definition of T14[TOI178d]=2.35 h
Default nsub_Dpl[TOI178d]=17

-----------------------
Processing instrument : ESPRESSO
-----------------------
  Reading and initializing CCFs
   > Data resampled on a common spectral table
         Retrieving data
           All exposures in Test_night share a common spectral table
   > 19 exposures
  -----------------
  Processing visit: Test_night
  -----------------
   > 19 exposures
         7 in-transit
         12 out-of-transit (6 pre / 6 post)
   > Calculating properties of planet-occulted regions
         Calculating data
   > Analyzing disk-integrated stellar profiles
         Calculating data
         Systemic rv from time-series = 0.000069 +- 1.634028e-04 km/s
   > Aligning disk-integrated prof

1 extra bytes in post.stringData array
'created' timestamp seems very low; regarding as unix timestamp
Zapf NOT subset; don't know how to subset; dropped
feat NOT subset; don't know how to subset; dropped
meta NOT subset; don't know how to subset; dropped
morx NOT subset; don't know how to subset; dropped


     ----------------------------------

-----------------------------------
Plots
-----------------------------------
Numerical T14[TOI178d]=2.345168 h
Numerical Kp_orb=98.26 km/s (Mstar = 0.645 Msun)
-----------------------------------
+ Individual disk-integrated profiles
      - Instrument : ESPRESSO
        - Visit : Test_night
-----------------------------------
+ Properties of raw data and disk-integrated profiles
   > RV
     ESPRESSO
      Test_night
   ---------------
   > Contrast
     ESPRESSO
      Test_night
   ---------------
   > FWHM
     ESPRESSO
      Test_night
   ---------------
-----------------------------------
+ Individual intrinsic profiles
      - Instrument : ESPRESSO
        - Visit : Test_night
Stop : No existing fit results


SystemExit: 

/Users/bourrier/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Plot display

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 

Run the cells below to show saved plots.

**Disk-integrated CCF series**

This plot shows the series of disk-integrated CCFs (blue profiles) with their best-fit Gaussian models (dashed black profiles), from which are derived the properties showed in the next plot.

In [ ]:
Image(filename = input_nbook['plot_path']+'DI_data/'+input_nbook['par']['instrument']+'_'+input_nbook['par']['night']+'_Indiv/Data/CCF/phase.gif')

**Disk-integrated line properties**

The three plots below show the time-series of RVs, contrast, and FWHM derived from the fits to the disk-integrated CCFs. Transit contacts are shown as dashed black lines, and in-transit exposures are flagged with empty symbols.

Outside of the transit, RVs should trace the Keplerian motion of your star with respect to the solar system barycenter (the Keplerian model defined by your inputs for the star and planets is plotted as a solid blue curve). If your star is quiet and measurements are dominated by white noise, RV measurements should be evenly spread around the Keplerian model, while contrast and FWHM measurements should be evenly spread around their mean out-of-transit value (dashed blue lines). You can assess this by looking at the ratio between the dispersion and mean error over out-of-transit measurements, reported on each plot, which should be close to unity. If that is not the case, you can use the workflow to detrend the data.  

Inside of the transit the planet occults local regions of the stellar surface, distorting the disk-integrated stellar lines and inducing anomalous deviations in their derived properties. Traditionally, measurements of the sky-projected spin-orbit angle in planetary systems have been performed by analyzing this anomalous signal in RV - but the anomaly may not be detectable with faint, slow-rotating stars and small planets. With ANTARESS, you will be able to analyze directly the planet-occulted stellar profiles to get a higher precision on the transit chord and orbital architecture of the system.

In [ ]:
Image(filename = input_nbook['plot_path']+'DI_prop/rv_phase.png')

In [ ]:
Image(filename = input_nbook['plot_path']+'DI_prop/ctrst_phase.png')

In [ ]:
Image(filename = input_nbook['plot_path']+'DI_prop/FWHM_phase.png')

**Intrinsic CCF series**

This plot shows the series of intrinsic CCFs (blue profiles) with their best-fit Gaussian models (dashed black profiles), from which are derived the properties showed in the next plots.

In [ ]:
Image(filename = input_nbook['plot_path']+'Intr_data/'+input_nbook['par']['instrument']+'_'+input_nbook['par']['night']+'_Indiv/Data/CCF/phase.gif')

**Intrinsic line properties**

The three plots below show the time-series of RVs, contrast, and FWHM derived from the fits to intrinsic CCFs, which directly trace the properties of the photosphere along the transit chord. Transit contacts are shown as dashed black lines.

The dashed orange lines show the best-fit models you derived for each property series. You can see how the decreasing intensity at the stellar limb, combined with its partial occultation by the planet, leads to intrinsic line properties being derived at a lower precision. For smaller planets and fainter stars all exposures may yield a poor fit individually, making it hazardous to interpret the property time series. Fortunately, useful information can still be retrieved from these exposures by fitting the time series of intrinsic line profiles with a joint model through a RM Revolutions fit. The dashed black lines show the properties associated with this best fit.

In [ ]:
Image(filename = input_nbook['plot_path']+'Intr_prop/rv_phase.png')

In [ ]:
Image(filename = input_nbook['plot_path']+'Intr_prop/ctrst_phase.png')

In [ ]:
Image(filename = input_nbook['plot_path']+'Intr_prop/FWHM_phase.png')

**Intrinsic line maps**



In [ ]:
Image(filename = input_nbook['plot_path']+'Intr_data/ESPRESSO_20230624_Map/Data/CCF/iord0.png')

In [ ]:
Image(filename = input_nbook['plot_path']+'Intr_data/ESPRESSO_20230624_Map/Model/CCF/iord0_glob_mod.png')

In [ ]:
Image(filename = input_nbook['plot_path']+'Intr_data/ESPRESSO_20230624_Map/Res/CCF/iord0_glob_mod_res.png')